# Final Project - Performance Analysis of File Transfer Protocols
### ECE-4400
### Carl Tuck, Henry Powers, Daniel Pacheco, Arya Patel
### 1 April 2025

## Setup configuration for Using Fabric Testbed via FABlib API

- `fabric_rc`:  File used to configure a FABlib application.  
- `fabric_bastion_key`: Fabric Bastion key pair. In order to minimize security incidents on FABRIC, access to VMs and other resources administered by users is controlled using a bastion host. You will need to set up an ssh keypair that will be used to jump through the bastion host to your VMs and other resources. This keypair is unique to you and is only used to set up ssh proxy connections through the bastion host to your FABRIC resources. More information about how to access your experiment through the bastion host can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
- `slice_key` and `slice_key.pub`: Sliver Key pair.
- `ssh_config`: File used to ssh from from a terminal to FABRIC VM by jumping through the FABRIC bastion host. 

## Set Project ID and generate the configuration

Edit the following cell by entering your Project ID for the FABRIC Project to use in your Jupyter container.  

- The Project ID can be from any of your projects. The ID can be found in the 'Basic Info' tab for each of the [projects](https://portal.fabric-testbed.net/experiments#projects) in the FABRIC portal.

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

# Update this line to specify your project id
project_id = "0508639f-d515-44c8-b922-cf4cf000b6d9"

# Uncomment the line below if using 'FABRIC Tutorials' Project
#project_id="a7818636-1fa1-4e77-bb03-d171598b0862"

fablib = fablib_manager(project_id=project_id)

## Display the configuration

In [2]:
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Bastion Host,bastion.fabric-testbed.net
Bastion Username,cetuck_0000291746
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


## Validate the configuration;
- Checks the validity of the bastion keys and regenerates them if they are expired
- Generates Sliver keys if they do not exist already

In [3]:
fablib.verify_and_configure()

User: cetuck@clemson.edu bastion key is valid!
Configuration is valid and please save the config!


## Save the configuration for subsequent use

In [4]:
fablib.save_config()

## Set up the Experiment
This section uses the Fablib manager to create a new slice that is composed of 2 nodes.

## Reserve Resources
In the 'CLEM' site, we will reserve a set of 2 nodes arranged in a line, with one node designated as a server and the other as a client. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. Each node will have 1 network card (NICs) to communicate with each other. All nodes will be preloaded with an 'Ubuntu' Linux OS. Upon submission, the slice will be named 'FinalProject'.

In [5]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Bastion Host,bastion.fabric-testbed.net
Bastion Username,cetuck_0000291746
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [6]:
# Define and Submit Slice
slice_name = "FinalProject"
site = "CLEM"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    client = slice.add_node(name="client", site=site)
    client.set_capacities(cores=cores, ram=ram, disk=disk)
    client.set_image(image)
    iface1 = client.add_component(model='NIC_Basic', name="client-nic1").get_interfaces()[0]
    
    # Add node
    server = slice.add_node(name="server", site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    iface2 = server.add_component(model='NIC_Basic', name="server-nic1").get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name="bridge1", interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")


Retry: 10, Time: 236 sec


ID,d196ce0a-1532-4a6f-a361-2ea8ee3c6c69
Name,FinalProject
Lease Expiration (UTC),2025-04-02 00:52:11 +0000
Lease Start (UTC),2025-04-01 00:52:11 +0000
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
186e9596-2381-4ec3-98b3-01cd949967f8,client,1,2,10,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fed2:693a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fed2:693a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e40b3e96-23e2-4c81-b022-a6cdeea5ca62,server,1,2,10,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fef0:c47,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fef0:c47,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
aff6a25c-709b-42d1-a7b9-f2c8eb34ea9a,bridge1,L2,L2Bridge,CLEM,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
client-client-nic1-p1,p1,client,bridge1,100,config,,0A:52:8F:34:96:C6,enp7s0,enp7s0,fe80::852:8fff:fe34:96c6,6,HundredGigE0/0/0/5
server-server-nic1-p1,p1,server,bridge1,100,config,,0A:56:AB:E2:68:40,enp7s0,enp7s0,fe80::856:abff:fee2:6840,6,HundredGigE0/0/0/5



Time to print interfaces 240 seconds


## Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "bridge1".

This slice contains a single subnetwork in IPv4: 10.20.30.0/24 (between server and the client, "bridge1"). This cell uses the alternative "cidr" to set the subnet based on the specified number of bits. This slice will provide IP 10.20.30.40 to the client node and 10.20.30.41 to the server node following the guidelines set by the network subnet.

After providing the IPs to the corresponding devices, the cell will verify that all interfaces are up and execute "ip a" to output the connections that are active. Then, we will add the additional route from the client to the server in case the route is not added correctly.

In [7]:
# Setup Network
try:
    client_name = "client"
    server_name = "server"
    network_service_name = "bridge1"

    client_ip = '10.20.30.40'
    server_ip = '10.20.30.41'
    
    client = slice.get_node(name=client_name)        
    iface1 = client.get_interface(network_name=network_service_name)  
    iface1.set_ip(ip=client_ip, cidr="24")
    
    iface1.get_device_name()
    
    client.execute("echo 100.20.30.40 client | sudo tee -a /etc/hosts")
    client.execute(f"sudo ip link set dev { iface1.get_device_name()} up")
    
    stdout, stderr = client.execute(f'ip addr show {iface1.get_os_interface()}')
    server = slice.get_node(name=server_name)        
    iface2 = server.get_interface(network_name=network_service_name)  
    iface2.set_ip(ip=server_ip, cidr="24")
    
    stdout, stderr = server.execute(f'ip addr show {iface2.get_os_interface()}')
    server.execute(f"sudo ip link set dev { iface2.get_device_name()} up")
 
    print("Printing Server's Network")
    out, err = server.execute("ip a ")
    
    print("Getting control interface ip")
    out, err = server.execute("ip a | grep inet | grep enp | awk '{print $2}'")
    ip = ((out.split('\n')[0]).split('/'))[0]
    print(f"Ip found: {ip}")
    
    print("Getting Client's interface name")
    print(f'Device Found: {iface1.get_device_name()}')
    
    # add route if needed
    client.execute(f"sudo ip route add {ip} dev {iface1.get_device_name()}")

except Exception as e:
    print(f"Error: {e}")

100.20.30.40 client
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:52:8f:34:96:c6 brd ff:ff:ff:ff:ff:ff
    inet 10.20.30.40/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::852:8fff:fe34:96c6/64 scope link 
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:56:ab:e2:68:40 brd ff:ff:ff:ff:ff:ff
    inet 10.20.30.41/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::856:abff:fee2:6840/64 scope link 
       valid_lft forever preferred_lft forever
Printing Server's Network
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_l

## Configure the software needed for the nodes in the experiment¶
In this section of the slice creation, we will add any additional software, tools or scripts that we need for our experiments. 

In [8]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    print("Installing required software packages...")
    
    # Get our nodes
    client = slice.get_node(name="client")
    server = slice.get_node(name="server")
    
    # Common packages for both nodes
    packages = [
        "openssh-server",  # For SCP/SFTP
        "vsftpd",         # For FTP server
        "iperf3",         # For bandwidth testing
        "wget",           # For FTP/HTTP downloads
        "net-tools"       # For network utilities
    ]
    
    # Update and install packages on both nodes
    for node in [client, server]:
        print(f"Updating packages on {node.get_name()}...")
        node.execute("sudo apt-get update -y")
        
        print(f"Installing packages on {node.get_name()}...")
        install_cmd = f"sudo apt-get install -y {' '.join(packages)}"
        node.execute(install_cmd)
    
    # Server-specific configuration
    print("Configuring FTP server on server node...")
    server.execute("sudo systemctl start vsftpd")
    server.execute("sudo systemctl enable vsftpd")
    
    # Create test directory on both nodes
    test_dir = "~/file_transfer_test"
    for node in [client, server]:
        node.execute(f"mkdir -p {test_dir}")
    
    # Generate test files on server (1MB, 10MB, 100MB)
    print("Generating test files on server...")
    server.execute(f"cd {test_dir} && dd if=/dev/urandom of=test_1MB.dat bs=1M count=1 status=none")
    server.execute(f"cd {test_dir} && dd if=/dev/urandom of=test_10MB.dat bs=1M count=10 status=none")
    server.execute(f"cd {test_dir} && dd if=/dev/urandom of=test_100MB.dat bs=1M count=100 status=none")
    
    print("Software installation and configuration complete!")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

Installing required software packages...
Updating packages on client...
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ub

## Retrieve Slice

### Import the Fabric API

In [9]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

try:
    slice_name = 'FinalProject'
    
    slice = fablib.get_slice(slice_name)
    slice.list_nodes()
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Bastion Host,bastion.fabric-testbed.net
Bastion Username,cetuck_0000291746
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
186e9596-2381-4ec3-98b3-01cd949967f8,client,1,2,10,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fed2:693a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fed2:693a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e40b3e96-23e2-4c81-b022-a6cdeea5ca62,server,1,2,10,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fef0:c47,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fef0:c47,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


Slice: FinalProject, StableOK


### View Topology

#### Get the topology
To configure the Nodes we need to get the topology of the experiment by getting the slice component. In this tutorial we get the slice by the name given when the slice was created.
The folowing cell prints the topology.

In [10]:
# Preview Slice node's attributes
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

Node:
   Name              : client
   Host              : clem-w1.fabric-testbed.net
   Site              : CLEM
   Management IP     : 2620:103:a006:12:f816:3eff:fed2:693a
   Reservation ID    : 186e9596-2381-4ec3-98b3-01cd949967f8
   Reservation State : Active
   Interfaces        : [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7c3c796499d0>]
   SSH Command       : ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fed2:693a

Node:
   Name              : server
   Host              : clem-w1.fabric-testbed.net
   Site              : CLEM
   Management IP     : 2620:103:a006:12:f816:3eff:fef0:c47
   Reservation ID    : e40b3e96-23e2-4c81-b022-a6cdeea5ca62
   Reservation State : Active
   Interfaces        : [<fabrictestbed_extensions.fablib.interface.Interface object at 0x7c3c794f8510>]
   SSH Command       : ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_co

# File Transfer Protocol Testing

## Log in to the Nodes

In the Retrieve Slice section above, get the ssh commands from the Nodes table.

Open two terminals, ssh in to the server node on one and the client node on the other. 

-----

## Set up SSH

### On the Client Node:

Generate an SSH key.

`ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N ""`

Display the SSH key.

`cat ~/.ssh/id_rsa.pub`

-----

### On the Server Node:

Make a directory for the SSH key.

`mkdir -p ~/.ssh`

Use this command, but replace what's in the quotes with the displayed SSH key from the client node command above: `cat ~/.ssh/id_rsa.pub`.

```bash
echo "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDJFnnAgBMP8LHxI2GVwwpZ4TwNCUBPIl4qflh8qrKzkqcGtTGc2BRZoL6kynfO0o0PcHb0U3agPWQr/92B6zpK/OybmImIpEC3KO796+JjMU+VSvlkmRfrkB0XZ1tpGwn4RskTC7W10flPwu0b/fMg358l6PmsApPsesmnEqkOix44Kmr1ySNUomLh+gbZak1OWb1ajshmApC8rqKVzFxvzV3pZ3VOFEfAWEOGLoHaKVkiU3AGwtCpYzkUcMGvUiJ5f4xtD0vjwFh0PKHVSJ5MpA0StCtOSh2hMPmXlkGFvZ1cNFaAWu5qyCE7fgw6bKeCxgbz13TX24o1LcJB32U8bu00iBJypEV6KO1lok5loF1znhLwAG+G+d4ZBi3rv/eFw1blnngzqi05Re/NHe5uZdb5YFJ3BgucsqA2XP0yggOsDiZ0LWiMp9Wjuv1JOqn5rY8EnjMlJtdXSHwjtbwsSUM1bqnjkgoNNzGJoDLiGJj7LoEFbH2LCBlOt4x59/NW2y1HFKDVu6hnVwnN76qAyQvi4NvNdhtVmgKTzvuTySh1DyRXcvElXiFT69uJFAgLaqF471rQsu31HCmuiR2x7uwULciarg2n5DB/WeJy9CatzHe3xqjYJAnwGV12oUiGNvRKuwAm0YIZ9YP1+BPWCBx7LK+iMFRYJI2jvWqpxw== ubuntu@client" >> ~/.ssh/authorized_keys
```

Run these two `chmod` commands:

`chmod 700 ~/.ssh`

`chmod 600 ~/.ssh/authorized_keys`

-----

### Back on the Client Node:

Test the SSH functionality with:

`ssh -i ~/.ssh/id_rsa ubuntu@10.20.30.41`

Exit the SSH session with:

`exit`

-----

## Begin Testing

### SCP 

#### On the Client Node:

```bash
mkdir -p ~/transfer_results
for size in 1 10 100; do
    echo "Testing SCP with ${size}MB file"
    { time scp -i ~/.ssh/id_rsa ~/file_transfer_test/test_${size}MB.dat ubuntu@10.20.30.41:~/ ; } 2>> ~/transfer_results/scp_times.txt
    echo "-----" >> ~/transfer_results/scp_times.txt
done
```

-----

### SFTP

#### On the Client Node:

```bash
for size in 1 10 100; do
    echo "Testing SFTP with ${size}MB file"
    { time sftp -i ~/.ssh/id_rsa -b <(echo "put test_${size}MB.dat") ubuntu@10.20.30.41 ; } 2>> ~/transfer_results/sftp_times.txt
    echo "-----" >> ~/transfer_results/sftp_times.txt
done
```

-----

### FTP

#### On the Server Node:

```bash
sudo apt install vsftpd
sudo sed -i 's/^#\?anonymous_enable=.*/anonymous_enable=YES/' /etc/vsftpd.conf
sudo sed -i 's/^#\?local_enable=.*/local_enable=YES/' /etc/vsftpd.conf
sudo sed -i 's/^#\?write_enable=.*/write_enable=YES/' /etc/vsftpd.conf
sudo systemctl restart vsftpd
```

#### On the Client Node:

```bash
for size in 1 10 100; do
    echo "Testing FTP with ${size}MB file"
    { time curl -T ~/file_transfer_test/test_${size}MB.dat ftp://10.20.30.41/ --user anonymous: ; } 2>> ~/transfer_results/ftp_times.txt
    echo "-----" >> ~/transfer_results/ftp_times.txt
done
```

-----

## Data Analysis

### On the Client Node:

```bash
# Process timing data
for protocol in scp sftp ftp; do
    echo "${protocol} results:"
    grep real ~/transfer_results/${protocol}_times.txt | awk '{print $2}' | xargs -n3 | awk '{print "1MB:", $1, "10MB:", $2, "100MB:", $3}'
done

# Calculate transfer speeds
for size in 1 10 100; do
    scp_time=$(grep -A1 "Testing SCP with ${size}MB" ~/transfer_results/scp_times.txt | grep real | awk '{print $2}')
    echo "SCP ${size}MB: $(echo "$size/$scp_time" | bc -l) MB/s"
done
```

-----

## Cleanup

#### On Both Nodes:

`rm -rf ~/file_transfer_test/test_*.dat`


## Cleanup Resources

In [11]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")

Change the Boolean to delete slice
